# 10

## 10.1 2値分類の性能指標

### 10.1.1 陽性と陰性

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y       = np.array([  0,   1,   1,   0,   1,   0,    1,   0,   0,   1])
y_score = np.array([0.7, 0.8, 0.3, 0.4, 0.9, 0.6, 0.99, 0.1, 0.2, 0.5])

In [ ]:
y_ = np.array([1 if 0.5 <= p else 0 for p in y_score])
y_
#> array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1])

In [ ]:
confusion_matrix(y_true=y, y_pred=y_)
#> array([[3, 2],
#>        [1, 4]])

print(classification_report(y_true=y, y_pred=y_))
#>               precision    recall  f1-score   support
#>
#>            0       0.75      0.60      0.67         5
#>            1       0.67      0.80      0.73         5
#>
#>     accuracy                           0.70        10
#>    macro avg       0.71      0.70      0.70        10
#> weighted avg       0.71      0.70      0.70        10

## 10.2 トレードオフ

### 10.2.1 偽陽性率と真陽性率のトレードオフ（ROC曲線）

In [ ]:
import numpy as np
from sklearn.metrics import (roc_curve, RocCurveDisplay,
    precision_recall_curve, PrecisionRecallDisplay, auc)

y       = np.array([  0,   1,   1,   0,   1,   0,    1,   0,   0,   1])
y_score = np.array([0.7, 0.8, 0.3, 0.4, 0.9, 0.6, 0.99, 0.1, 0.2, 0.5])
y_      = np.array([1 if 0.5 <= p else 0 for p in y_score])

[sum((y == 0) & (y_ == 1)) / sum(y == 0), # FPR
 sum((y == 1) & (y_ == 1)) / sum(y == 1)] # TPR
#> [0.4, 0.8]

In [ ]:
my_fpr, my_tpr, _ = roc_curve(y_true=y,
                              y_score=y_score,
                              pos_label=1) # 1が陽性である．
RocCurveDisplay(fpr=my_fpr, tpr=my_tpr).plot()

In [ ]:
auc(x=my_fpr, y=my_tpr)
#> 0.8

### 10.2.2 再現率と精度のトレードオフ（PR曲線）

In [ ]:
[sum((y == 1) & (y_ == 1)) / sum(y  == 1), # Recall == TPR
 sum((y == 1) & (y_ == 1)) / sum(y_ == 1)] # Precision
#> [0.8, 0.6666666666666666]

In [ ]:
my_precision, my_recall, _ = precision_recall_curve(y_true=y,
                                                    probas_pred=y_score,
                                                    pos_label=1)
PrecisionRecallDisplay(precision=my_precision, recall=my_recall).plot()

In [ ]:
auc(x=my_recall, y=my_precision)
#> 0.8463095238095237

## 10.3 タイタニック

In [ ]:
import graphviz
import pandas as pd
from sklearn import tree
from sklearn.metrics import roc_curve, RocCurveDisplay, auc
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

my_url = ('https://raw.githubusercontent.com/taroyabuki'
          '/fromzero/master/data/titanic.csv')
my_data = pd.read_csv(my_url)

In [ ]:
my_data.head()
#>   Class   Sex    Age Survived
#> 0   1st  Male  Child      Yes
#> 1   1st  Male  Child      Yes
#> 2   1st  Male  Child      Yes
#> 3   1st  Male  Child      Yes
#> 4   1st  Male  Child      Yes

### 10.3.1 質的入力変数の扱い方

### 10.3.2 決定木の訓練

In [ ]:
X, y = my_data.iloc[:, 0:3], my_data.Survived

my_pipeline = Pipeline([
    ('ohe', OneHotEncoder(drop='first')),
    ('tree', tree.DecisionTreeClassifier(max_depth=2, random_state=0,
                                         min_impurity_decrease=0.01))])
my_pipeline.fit(X, y)

### 10.3.3 決定木の描画

In [ ]:
my_enc  = my_pipeline.named_steps['ohe']  # パイプラインからエンコーダを取り出す．
my_tree = my_pipeline.named_steps['tree'] # パイプラインから木を取り出す．

my_dot = tree.export_graphviz(
    decision_tree=my_tree,
    out_file=None,
    feature_names=my_enc.get_feature_names() \
    if hasattr(my_enc, 'get_feature_names') else my_enc.get_feature_names_out(),
    class_names=my_pipeline.classes_,
    filled=True)
graphviz.Source(my_dot)

### 10.3.4 決定木の評価

In [ ]:
my_scores = cross_val_score(
    my_pipeline, X, y,
    cv=LeaveOneOut(),
    n_jobs=-1)
my_scores.mean()
#> 0.7832803271240345

In [ ]:
tmp = pd.DataFrame(
    my_pipeline.predict_proba(X),
    columns=my_pipeline.classes_)
y_score = tmp.Yes

my_fpr, my_tpr, _ = roc_curve(y_true=y,
                              y_score=y_score,
                              pos_label='Yes')
my_auc = auc(x=my_fpr, y=my_tpr)
my_auc
#> 0.7114886868858494

RocCurveDisplay(fpr=my_fpr, tpr=my_tpr, roc_auc=my_auc).plot()

### 10.3.5 補足：質的入力変数の扱い

## 10.4 ロジスティック回帰

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(-6, 6, 0.1)
y = 1 / (1 + np.exp(-x))
plt.plot(x, y)

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

my_url = ('https://raw.githubusercontent.com/taroyabuki'
          '/fromzero/master/data/titanic.csv')
my_data = pd.read_csv(my_url)

X, y = my_data.iloc[:, 0:3], my_data.Survived

my_pipeline = Pipeline([('ohe', OneHotEncoder(drop='first')),
                        ('lr', LogisticRegression(penalty='none'))])
my_pipeline.fit(X, y)

In [ ]:
my_ohe = my_pipeline.named_steps.ohe
my_lr  = my_pipeline.named_steps.lr

my_lr.intercept_[0]
#> 2.043878162056783

tmp = my_ohe.get_feature_names() \
if hasattr(my_ohe, 'get_feature_names') \
else my_ohe.get_feature_names_out()
pd.Series(my_lr.coef_[0],
          index=tmp)
#> x0_2nd     -1.018069
#> x0_3rd     -1.777746
#> x0_Crew    -0.857708
#> x1_Male    -2.420090
#> x2_Child    1.061531
#> dtype: float64

In [ ]:
my_scores = cross_val_score(
    my_pipeline, X, y,
    cv=LeaveOneOut(),
    n_jobs=-1)
my_scores.mean()
#> 0.7782825988187188